In [ ]:
import urllib
import sqlalchemy
import numpy as np
import pandas as pd
from datetime import datetime

import os
import pandas as pd
import numpy as np


from helpers import rutas, validarProduccion
rutaRelativa        = rutas.rutas()
print(rutaRelativa)
estadoProduccion    = validarProduccion.validarProduccion()


from classes import dbAzure, email
dbAzure = dbAzure.DBAzure(estadoProduccion)
Email   = email.Email

print(estadoProduccion)

In [ ]:
# CREAR HISTORICO FRIO DS
def queryCreateTableHistorico_equipos_DS(conection_DB, nombre_tabla, dataframe):
    
    # ELIMINA BASE DE DATOS
    conection_DB.connection.execute(f'DROP TABLE GestionProcesosTienda.dbo.{nombre_tabla};')
    conection_DB.connection.commit()
    
    # CREA ARREGLO CON LAS COLUMNAS Y SUS TIPOS
    columns = []
    for index in dataframe.dtypes.index:
        tipo = str(dataframe.dtypes.loc[index])
        if tipo == "object":
            tipo = 'varchar'
        elif tipo == "int64":
            tipo = 'int'
        elif tipo == "float64":
            tipo = 'float'
        columns.append({'name': index, 'type': tipo})
    
    print(columns)
    
    # DA FORMATO A QUERY PARA CREAR TABLA 
    query = f"CREATE TABLE GestionProcesosTienda.dbo.{nombre_tabla} (\nid bigint IDENTITY(1,1) NOT NULL,"
    for column in columns:
        if column['type'] == 'varchar':
            query += f'\n{column["name"]} varchar(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL,'
        elif column['type'] == 'int':
            query += f'\n{column["name"]} bigint NOT NULL,'
    query += f"\ncreated_at datetime NOT NULL, \nupdated_at datetime NOT NULL, \nCONSTRAINT PK_{nombre_tabla}_3213E83F1717089E PRIMARY KEY (id)\n);"

    # SI LA TABLA NO EXISTE EN LA BASE DE DATOS LA CREA
    tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
    if 'historico_equipos_frio_DS' not in tablas.TABLE_NAME.unique():
        # CREA TABLA
        conection_DB.connection.execute(query)
        conection_DB.connection.commit()

        # AGREGA LAS COLUMNAS DE FECHA DE CREACION Y ACTUALIZACION
        dataframe['created_at']   = datetime.today()
        dataframe['updated_at']   = datetime.today()

        # INSERTA DATOS EN BASE DE DATOS
        db_params               = urllib.parse.quote_plus(conection_DB.dataConnection)
        engine                  = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
        dataframe.to_sql('historico_equipos_frio_DS', con=engine, index=False, if_exists="append", schema="dbo")

In [ ]:
display(f'{rutaRelativa}/ROTACION/temperatura_diarias.cvs')

In [ ]:
dataframe = pd.read_csv(f'{rutaRelativa}/ROTACION/temperatura_diarias.csv')
dataframe = dataframe
#dataframe = dataframe.drop(['EAN','SECCION','ALMACENAMIENTO','Fecha Merma','SECCION','N° Doc. Merma','ZONA','Creado por','Modificado','Creado','NO_CONFORMIDAD','CANTIDAD','ESTADO_RETIRO','FECHA_RETIRO','DESTINO','MES','ID_SECCION','LIMPIEZA','FECHA_SOLICITUD','PROVEEDOR','UBICACION','FECHA_VENCIMIENTO' ], axis=1)
dataframe = dataframe.replace({np.nan: 'None'})
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: pd.Series(str(x).split(".")[0]))
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: '0' + x if(int(x) < 10) else x)

# MODIFICAR NOMBRES DE COLUMAS
dataframe = dataframe.drop('AC2', axis=1)

dataframe = dataframe.rename({
    'Creado por'    :'Creado_por'
    },axis=1)

#for i in dataframe.index:
#    print(dataframe['Creado'])
#
#for i in dataframe.index:
#    print(dataframe.iloc[1]['Creado'])
#    dataframe.iloc[i]['ANO'] = i+1
#    print(dataframe.iloc[i]['ANO'])

dataframe['ANO'] = 2020;
dataframe['MES'] = 1;
dataframe['DIA'] = 27;
dataframe['CAMBIO_ESTADO'] = 'None'
dataframe['nombreProducto2'] = ''
dataframe['temperaturaProducto2'] = ''
dataframe['CANTIDAD_FALLOS'] = '0'

dataframe['nombreMaquina'] = dataframe["nombreMaquina"].apply(lambda x: pd.Series(str(x).split(".")[0]))

display(dataframe)
dataframe.dtypes

In [ ]:
display(dataframe)
dataframe.dtypes

In [ ]:
# EJECUTA FUNCION HISTORITO EQUIPOS FRIO DS
nombre_tabla = 'historico_equipos_frio_DS'
queryCreateTableHistorico_equipos_DS(dbAzure, nombre_tabla, dataframe)